#1.Se importan las librerías

In [51]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
import os

#2.Se cargan los datos de mnist

In [52]:
#Se cargan los datos y se define tanto el conjunto de entrenamiento como de test
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

#3.Se monta la unidad de drive 

In [17]:
#Se monta la unidad de drive con el fin de extraer los datos que almacenamos en nuestro Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#4.Se declara la clase CNNBlock

In [53]:
class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding="same")
        self.bn = layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        return x

#5.Se declara la clase ResBlock

In [54]:
class ResBlock(layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.channels = channels
        self.cnn1 = CNNBlock(channels[0], 3)
        self.cnn2 = CNNBlock(channels[1], 3)
        self.cnn3 = CNNBlock(channels[2], 3)
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], 3, padding="same")

    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor), training=training,)
        x = self.pooling(x)
        return x

#6.Se declara la clase ResNet_Like

In [55]:
class ResNet_Like(keras.Model):
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__()
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([128, 128, 256])
        self.block3 = ResBlock([128, 256, 512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)

    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x, training=training)
        x = self.classifier(x)
        return x

    def model(self):
        x = keras.Input(shape=(28, 28, 1))
        return keras.Model(inputs=[x], outputs=self.call(x))


#7.Se instancia el modelo ResNet_Like

In [40]:

model = ResNet_Like().model()
base_input = model.layers[0].input
base_output = model.layers[2].output
output = layers.Dense(10)(layers.Flatten()(base_output))
model = keras.Model(base_input, output)


#8.Se definen los parámetros del modelo 

In [41]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

#9. Se valida que haya un GPU para entrenar

In [46]:
nombre_gpu = tf.test.gpu_device_name()
if nombre_gpu != '/device:GPU:0':
  raise SystemError('GPU no encontrada')
print('GPU encontrada: {}'.format(nombre_gpu))

GPU encontrada: /device:GPU:0


#10. Se entrena el modelo 

In [48]:
import timeit

def entrenamiento_gpu():
  with tf.device('/device:GPU:0'):
    model.fit(x_train, y_train, batch_size=64, epochs=1, verbose=2)
  
  return None

gpu_time = timeit.timeit('entrenamiento_gpu()', number=1, setup='from __main__ import entrenamiento_gpu')

938/938 - 17s - loss: 0.0402 - accuracy: 0.9881 - 17s/epoch - 18ms/step


#11.Evaluación del modelo en el conjunto de test 

In [49]:
model.evaluate(x_test, y_test, batch_size=64, verbose=2)


157/157 - 1s - loss: 0.0274 - accuracy: 0.9905 - 1s/epoch - 8ms/step


[0.027422567829489708, 0.9904999732971191]

#12. Se guarda el modelo 

In [50]:
model.save("/content/drive/MyDrive/Introduccion_Inteligencia_Artificial/Tarea_Vision_Artificial/Taller_Vision_Artificial_3/Resnet/pretrained")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Introduccion_Inteligencia_Artificial/Tarea_Vision_Artificial/Taller_Vision_Artificial_3/Resnet/pretrained/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Introduccion_Inteligencia_Artificial/Tarea_Vision_Artificial/Taller_Vision_Artificial_3/Resnet/pretrained/assets
